# Урок 3. Домашнее задание

1. Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете fashion.mnist. Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?

а) Исходный вариант

3 слоя: 28*28 + 128 + 10, 3 эпохи Асс= 0.877, Асс(тест)= 0.860

б) Попытка улучшить точность предсказаний за счет числа эпох улучшила показатель на тренировочном наборе, а на тестовом - не так сильно:

3 слоя: 28*28 + 128 + 10, 5 эпох Асс= 0.890, Асс(тест)= 0.867

в) Тот же прием, но для расширенного внутреннего слоя

3 слоя: 28*28 + 198 + 10, 3 эпохи Асс= 0.878, Асс(тест)= 0.871


3 слоя: 28*28 + 198 + 10, 5 эпох Асс= 0.893, Асс(тест)= 0.880

Вывод: уточнения результата можно добиться, одновременно увеличивая и число нейронов и число итераций

г) Попробуем завести слоев побольше и проведем 3 расчета с увеличением количества итераций:

5 слоев: 28*28 + 196 + 120 + 80 + 10, 3 эпохи Асс= 0.877, Асс(тест)= 0.864

5 слоев: 28*28 + 196 + 120 + 80 + 10, 5 эпох Асс= 0.890, Асс(тест)= 0.872

5 слоев: 28*28 + 196 + 120 + 80 + 10, 8 эпох Асс= 0.905, Асс(тест)= 0.871

Вывод: На тренировочном наборе удалось улучшить точность предсказаний, но тестовый набор показал, что наступает переобучение. У данной тактики (экстенсивное увеличение) обнаружен предел... Надо искать другие тактики


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Preprocess the data

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

### Определение слоев

Cоздаём 3 слоя нейронной сети с помощью функционала Keras.layers

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
#   keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(196, activation='relu'),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense( 80, activation='relu'),

    keras.layers.Dense(10)
])

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs=8)

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [0]:
predictions = probability_model.predict(test_images)

In [0]:
predictions[0]

In [0]:
np.argmax(predictions[0])

In [0]:
test_labels[0]

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Использование полученной модели

Давайте возьмем одно изображение из тестовой выборке и посмотрим предсказание нейронной сети - 

In [0]:
img = test_images[1]
print(img.shape)

In [0]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))
print(img.shape)

In [0]:
predictions_single = probability_model.predict(img)
print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`keras.Model.predict` возвращает список списков — по одному списку для каждого предсказания в батче. Нам нужны предсказания только для одного изображения:

In [0]:
np.argmax(predictions_single[0])

### Практическое задание

<ol>
   2. Поработайте с документацией TensorFlow 2. Найдите полезные команды не разобранные на уроке.
    
    
</ol>

Полезные команды: Символическое дифференцирование, автоматическое дифференцирование. Регуляризация через отключение нейронов (dropout) 